In [36]:
import os
import numpy as np
import pandas as pd
import matplotlib
import matplotlib.pyplot as plt
from sklearn.model_selection import KFold, train_test_split
from sklearn.metrics import accuracy_score, confusion_matrix, matthews_corrcoef, plot_confusion_matrix, f1_score
from sklearn.ensemble import GradientBoostingClassifier as gbc

import lightgbm as lgb
from lightgbm import LGBMClassifier

In [37]:
# read data
# basic feature 
bs_train = pd.read_csv('../input/ml4qsdata/data_initial/train.csv').drop(['Downstairs_label', 'Upstairs_label', 'Sitting_label', 'Walking_label', 'Standing_label', 'Jogging_label', 'Downstairs_label'], axis=1)
bs_test1 = pd.read_csv('../input/ml4qsdata/data_initial/test1.csv').drop(['Downstairs_label', 'Upstairs_label', 'Sitting_label', 'Walking_label', 'Standing_label', 'Jogging_label', 'Downstairs_label'], axis=1)
bs_test2 = pd.read_csv('../input/ml4qsdata/data_initial/test2.csv').drop(['Downstairs_label', 'Upstairs_label', 'Sitting_label', 'Walking_label', 'Standing_label', 'Jogging_label', 'Downstairs_label'], axis=1)

# data_selected
sl1_train = pd.read_csv('../input/ml4qsdata/data_selected/train.csv')
sl1_test1 = pd.read_csv('../input/ml4qsdata/data_selected/test1.csv')
sl1_test2 = pd.read_csv('../input/ml4qsdata/data_selected/test2.csv')

# 
sl21_train = pd.read_csv('../input/ml4qsdata/data_selected_21/train.csv')
sl21_test1 = pd.read_csv('../input/ml4qsdata/data_selected_21/test1.csv')
sl21_test2 = pd.read_csv('../input/ml4qsdata/data_selected_21/test2.csv')

In [38]:
bs_test1.head()

# Models

In [39]:
def gbc_(train, test1, test2):
    y = train.label
    X = train.drop('label', axis=1)
    
    y_test1 = test1.label
    X_test1 = test1.drop('label', axis=1)
    
    y_test2 = test2.label
    X_test2 = test2.drop('label', axis=1)
    
    kf = KFold(n_splits=5, shuffle=True)
    gbc_model = gbc()
    MMC = []
    oof1 = []
    oof2 = []
    acc1 = []
    acc2 = []
    for train_index, test_index in kf.split(X):
        X_train, X_test = X.loc[train_index], X.loc[test_index]
        y_train, y_test = y.loc[train_index], y.loc[test_index]
        gbc_model.fit(X_train, y_train)
        y_pred = gbc_model.predict(X_test)
        print(f"F1: {f1_score(y_test, y_pred, average='macro'):.3f}")
        MMC.append(f1_score(y_test, y_pred, average='macro'))
    
        y_test1_pred = gbc_model.predict(X_test1)
        oof1.append(f1_score(y_test1, y_test1_pred, average='macro'))
        acc1.append(accuracy_score(y_test1, y_test1_pred))
    
        y_test2_pred = gbc_model.predict(X_test2)
        oof2.append(f1_score(y_test2, y_test2_pred, average='macro'))
        acc2.append(accuracy_score(y_test2, y_test2_pred))
        
    return np.mean(oof1), np.mean(oof2), np.mean(acc1), np.mean(acc2), gbc_model

In [40]:
def lgbm_(train, test1, test2):
    y = train.label
    X = train.drop('label', axis=1)
    
    y_test1 = test1.label
    X_test1 = test1.drop('label', axis=1)
    
    y_test2 = test2.label
    X_test2 = test2.drop('label', axis=1)
    
    kf = KFold(n_splits=5, shuffle=True)
    lgbm = LGBMClassifier(n_estimators=5000)
    MMC = []
    oof1 = []
    oof2 = []
    acc1 = []
    acc2 = []
    for train_index, test_index in kf.split(X):
        X_train, X_test = X.loc[train_index], X.loc[test_index]
        y_train, y_test = y.loc[train_index], y.loc[test_index]
        lgbm.fit(X_train, y_train)
        y_pred = lgbm.predict(X_test)
        print(f"F1: {f1_score(y_test, y_pred, average='macro'):.3f}")
        MMC.append(f1_score(y_test, y_pred, average='macro'))
    
        y_test1_pred = lgbm.predict(X_test1)
        oof1.append(f1_score(y_test1, y_test1_pred, average='macro'))
        acc1.append(accuracy_score(y_test1, y_test1_pred))
    
        y_test2_pred = lgbm.predict(X_test2)
        oof2.append(f1_score(y_test2, y_test2_pred, average='macro'))
        acc2.append(accuracy_score(y_test2, y_test2_pred))
        
    print(lgbm)
        
    return np.mean(oof1), np.mean(oof2), np.mean(acc1), np.mean(acc2), lgbm

In [41]:
from sklearn import tree
def dtree_(train, test1, test2):
    y = train.label
    X = train.drop('label', axis=1)
    
    y_test1 = test1.label
    X_test1 = test1.drop('label', axis=1)
    
    y_test2 = test2.label
    X_test2 = test2.drop('label', axis=1)
    
    kf = KFold(n_splits=5, shuffle=True)
    dtree = tree.DecisionTreeClassifier(criterion='gini')
    MMC = []
    oof1 = []
    oof2 = []
    acc1 = []
    acc2 = []
    for train_index, test_index in kf.split(X):
        X_train, X_test = X.loc[train_index], X.loc[test_index]
        y_train, y_test = y.loc[train_index], y.loc[test_index]
        dtree.fit(X_train, y_train)
        y_pred = dtree.predict(X_test)
        print(f"F1: {f1_score(y_test, y_pred, average='macro'):.3f}")
        MMC.append(f1_score(y_test, y_pred, average='macro'))
    
        y_test1_pred = dtree.predict(X_test1)
        oof1.append(f1_score(y_test1, y_test1_pred, average='macro'))
        acc1.append(accuracy_score(y_test1, y_test1_pred))
    
        y_test2_pred = dtree.predict(X_test2)
        oof2.append(f1_score(y_test2, y_test2_pred, average='macro'))
        acc2.append(accuracy_score(y_test2, y_test2_pred))
        
        
    return np.mean(oof1), np.mean(oof2), np.mean(acc1), np.mean(acc2), dtree

In [42]:
f1_1, f1_2, acc_1, acc_2, model = dtree_(bs_train, bs_test1, bs_test2)
print("f1_test1: ", f1_1, " f1_test2: ", f1_2, " acc_test1: ", acc_1, " acc_test2: ", acc_2)

In [43]:
# Basic Features

In [44]:
f1_1, f1_2, acc_1, acc_2, model = gbc_(train = bs_train, test1 = bs_test1, test2 = bs_test2)
print("f1_test1: ", f1_1, " f1_test2: ", f1_2, " acc_test1: ", acc_1, " acc_test2: ", acc_2)

In [45]:
model

In [46]:
f1_1, f1_2, acc_1, acc_2, model = lgbm_(bs_train, bs_test1, bs_test2)
print("f1_test1: ", f1_1, " f1_test2: ", f1_2, " acc_test1: ", acc_1, " acc_test2: ", acc_2)

In [47]:
import shap
shap_values = shap.TreeExplainer(model.booster_).shap_values(bs_train.drop('label', axis=1))
shap.summary_plot(shap_values, bs_train.drop('label', axis=1))

# acc_userAcceleration.y_temp_std_ws_30 mag_rot

# Selected features

## gbc

In [48]:
f1_1, f1_2, acc_1, acc_2, model = gbc_(train = sl1_train, test1 = sl1_test1, test2 = sl1_test2)
print("f1_test1: ", f1_1, " f1_test2: ", f1_2, " acc_test1: ", acc_1, " acc_test2: ", acc_2)

## lightgbm

In [49]:
f1_1, f1_2, acc_1, acc_2, model = lgbm_(train = sl1_train, test1 = sl1_test1, test2 = sl1_test2)
print("f1_test1: ", f1_1, " f1_test2: ", f1_2, " acc_test1: ", acc_1, " acc_test2: ", acc_2)

In [50]:
import shap
shap_values = shap.TreeExplainer(model.booster_).shap_values(sl1_train.drop('label', axis=1))
shap.summary_plot(shap_values, sl1_train.drop('label', axis=1))

# Selected 21 feature 

## gbc

In [51]:
f1_1, f1_2, acc_1, acc_2, model = gbc_(train = sl21_train, test1 = sl21_test1, test2 = sl21_test2)
print("f1_test1: ", f1_1, " f1_test2: ", f1_2, " acc_test1: ", acc_1, " acc_test2: ", acc_2)

## lightgbm

In [52]:
f1_1, f1_2, acc_1, acc_2, model = lgbm_(train = sl21_train, test1 = sl21_test1, test2 = sl21_test2)
print("f1_test1: ", f1_1, " f1_test2: ", f1_2, " acc_test1: ", acc_1, " acc_test2: ", acc_2)

# Result

|                    | test1  |            | test2      |            |
|--------------------|------------|------------|------------|------------|
|                    | F1         | ACC        | F1         | ACC        |
| gbc [basic]        | 0.8667     | 0.8973     | 0.9176     | 0.9443     |
| lgbm [basic]       | **0.9110** | **0.9316** | **0.9690** | **0.9780** |
| gbc [selected 1]   | 0.5051     | 0.5958     | 0.6209     | 0.7204     |
| lgbm [selected 1]  | 0.5085     | 0.5737     | 0.7336     | 0.8032     |
| gbc [selected 21]  | 0.8578     | 0.8854     | 0.9138     | 0.9383     |
| lgbm [selected 21] | 0.8958     | 0.9162     | 0.9628     | 0.9734     |

% Please add the following required packages to your document preamble:
% \usepackage{multirow}
\begin{table}[]
\begin{tabular}{|l|ll|ll|}
\hline
\multirow{2}{*}{}      & \multicolumn{2}{l|}{test1}           & \multicolumn{2}{l|}{test2}           \\ \cline{2-5} 
                       & \multicolumn{1}{l|}{F1}     & ACC    & \multicolumn{1}{l|}{F1}     & ACC    \\ \hline
gbc {[}basic{]}        & \multicolumn{1}{l|}{0.8667} & 0.8973 & \multicolumn{1}{l|}{0.9176} & 0.9443 \\ \hline
lgbm {[}basic{]}       & \multicolumn{1}{l|}{0.9110} & 0.9316 & \multicolumn{1}{l|}{0.9690} & 0.9780 \\ \hline
gbc {[}selected 1{]}   & \multicolumn{1}{l|}{0.5051} & 0.5958 & \multicolumn{1}{l|}{0.6209} & 0.7204 \\ \hline
lgbm {[}selected 1{]}  & \multicolumn{1}{l|}{0.5085} & 0.5737 & \multicolumn{1}{l|}{0.7336} & 0.8032 \\ \hline
gbc {[}selected 21{]}  & \multicolumn{1}{l|}{0.8578} & 0.8854 & \multicolumn{1}{l|}{0.9138} & 0.9383 \\ \hline
lgbm {[}selected 21{]} & \multicolumn{1}{l|}{0.8958} & 0.9162 & \multicolumn{1}{l|}{0.9628} & 0.9734 \\ \hline
\end{tabular}
\end{table}